In [3]:
import os
# Change native directory to root
os.chdir(os.path.dirname(os.getcwd()))

In [4]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [9]:
data_test = pd.read_csv('data/extra_test.csv', index_col=0)
TARGET = 'net_target'
z = data_test[TARGET].values

In [10]:
z

array([0.40918536, 0.40918536, 0.40918536, ..., 0.49945068, 0.49982259,
       0.46495915])

In [22]:
def calc_covariance_matrix(alpha, sigma, horizon):
    """
    Calculate the covariance matrix for a given horizon
    """
    p = len(alpha)
    cov = np.zeros((horizon, horizon))
    for i in range(horizon):
        for j in range(horizon):
            if i <= j:
                cov[i, j] = sigma**2 if i == j else sigma**2 * np.sum(np.array([alpha[k] for k in range(p) if i-k >= 0 and i-k <= j-k]))
                if i != j:
                    cov[j, i] = cov[i, j]
    return cov

def log_likelihood(params, z, horizon):
    """
    Calculate the log-likelihood for the given parameters and time series data
    """
    alpha = params[:-1]
    sigma = params[-1]
    cov = calc_covariance_matrix(alpha, sigma, horizon)
    print(cov)
    inv_cov = np.linalg.inv(cov)
    logdet_cov = np.linalg.slogdet(cov)[1]
    N = z.shape[0]
    log_like = 0
    print('hello')
    for i in range(0, N-horizon+1, horizon):
        chunk = z[i:i+horizon]
        log_like += -0.5 * (logdet_cov + np.dot(chunk, np.dot(inv_cov, chunk)) + horizon * np.log(2 * np.pi))
    return -log_like

def estimate_params(z, horizon):
    """
    Estimate the parameters for the autoregressive process
    """
    p = 3
    init_params = np.zeros(p + 1)
    result = minimize(fun=log_likelihood, x0=init_params, args=(z, horizon))
    return result.x


In [24]:
p = 3
np.zeros(p + 1)

array([0., 0., 0., 0.])

In [23]:
# Example usage
#z = np.random.randn(8710) # generate some random time series data
horizon = 24
params = estimate_params(z, horizon)
alpha = params[:-1]
sigma = params[-1]
cov = calc_covariance_matrix(alpha, sigma, horizon)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.

LinAlgError: Singular matrix

In [42]:
# Define the order of the AR(p) process
p = 2

# Define the initial values for the parameters
initial_params = [1, 0.5, 0.5]

# Define the bounds for the parameters
bounds = [(1e-5, None), (-1, 1), (-1, 1)]

# Perform the optimization
res = minimize(log_likelihood, initial_params, args=(y, p), bounds=bounds, method='L-BFGS-B')

ValueError: shapes (8710,) and (24,24) not aligned: 8710 (dim 0) != 24 (dim 0)

In [33]:
# check if there is nan in y
np.isnan(y).any()

False